# 4.3 Анализ данных в индустрии. Проект 1 
## Карапетян Эдуард Каренович


#### Для того чтобы отслеживать, как выглядит путь клиента от знакомства с продуктом до его получения, и какие в нем есть проблемные места, аналитики часто строят воронки. Этот инструмент позволяет понять, сколько клиентов "теряется" на каждом из этапов процесса и сколько в итоге совершает целевое действие.

- Построить воронку, которая показывала бы в динамике по неделям и по месяцам
 - сколько клиентов посещает наш сайт, и какая доля из них
   - регистрируется на нем
   - подаёт заявку на игру
   - по факту приходит на игру.
 
Количество клиентов и конверсии необходимо изобразить на одном графике.


Как выполнять задание:

- Подключиться к базе данных с помощью Python в JupyterNotebook \ Google Colab 
- Собрать датасет с необходимыми полями,
- Посчитать конверсии и сделать визуализацию на Python.
- Прикрепить файл с выполненным заданием

### Вводим данные о нашей БД

In [12]:
import psycopg2
import pandas as pd

# Connection parameters
param_dic = {
    "host"      : "158.160.52.106",
    "database"  : "postgres",
    "user"      : "student",
    "password"  : "JvLda93aA"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [13]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
   
    tupples = cursor.fetchall()
    cursor.close()
    
    
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Вводим данные о нашей БД

In [14]:
# Connect to the database
conn = connect(param_dic)

column_names = ["date", "month", "month_title", "week","first_day_of_week", "year","daily_visits", "account_created", "application_created", "games", "visits_by_month","visits_by_week", "account_created_by_month", "account_created_by_week", "application_created_by_month", "application_created_by_week","games_by_month", "games_by_week", "visits_total_by_date", "account_created_total_by_date", "application_created_total_by_date", "games_total_by_date"]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "with client as (select visit_dttm::date as visit_date, count(distinct client_rk) as visits from msu_analytics.client c  group by 1), account as (select registration_dttm::date as acc_created_date, count(distinct account_rk) as account_created from msu_analytics.account a   group by 1), application as (select application_dttm::date as app_created_date, count(distinct application_rk) as application_created from msu_analytics.application	group by 1), games as (select game_dttm::date as game_date, sum(game_flg) as games	from msu_analytics.game group by 1	having sum(game_flg) >0) select visit_date as date, date_part('month',visit_date) as month,		case 			when date_part('month',visit_date) = 1 then 'January'	when date_part('month',visit_date) = 2 then 'February'			when date_part('month',visit_date) = 3 then 'March'			when date_part('month',visit_date) = 4 then 'April'			when date_part('month',visit_date) = 5 then 'May'			when date_part('month',visit_date) = 6 then 'June'			when date_part('month',visit_date) = 7 then 'July'			when date_part('month',visit_date) = 8 then 'August'			when date_part('month',visit_date) = 9 then 'September'			when date_part('month',visit_date) = 10 then 'October'			when date_part('month',visit_date) = 11 then 'November'			when date_part('month',visit_date) = 12 then 'December'		end as month_title,		date_part('week',visit_date) as week, DATE_TRUNC('week', visit_date)::date	as first_day_of_week,		date_part('year',visit_date) as year,		visits, coalesce (account_created,0) as account_created, coalesce (application_created,0) as application_created, coalesce (games, 0) as games,		sum(visits) over (partition by date_part('month',visit_date)) visits_by_month,		sum(visits) over (partition by date_part('week',visit_date)) visits_by_week, 		sum(account_created) over (partition by date_part('month',visit_date)) account_created_by_month,		sum(account_created) over (partition by date_part('week',visit_date)) account_created_by_week,		sum(application_created) over (partition by date_part('month',visit_date)) application_created_by_month,		coalesce (sum(application_created) over (partition by date_part('week',visit_date)),0) application_created_by_week,		coalesce (sum(games) over (partition by date_part('month',visit_date)),0) games_by_month,		coalesce (sum(games) over (partition by date_part('week',visit_date)),0) games_by_week,sum(visits) over (order by visit_date) as visits_total_by_date, sum(account_created) over (order by visit_date)	as account_created_total_by_date, sum(application_created) over (order by visit_date) as application_created_total_by_date,	sum(games) over (order by visit_date)	as games_total_by_date from client as c full join account a on a.acc_created_date = c.visit_date full join application app on app.app_created_date = c.visit_date full join games g on g.game_date = c.visit_date order by 1", column_names)
df.head()

Connecting to the PostgreSQL database...
Connection successful


,date,month,month_title,week,first_day_of_week,year,daily_visits,account_created,application_created,games,...,account_created_by_month,account_created_by_week,application_created_by_month,application_created_by_week,games_by_month,games_by_week,visits_total_by_date,account_created_total_by_date,application_created_total_by_date,games_total_by_date
0,2022-09-01,9.0,September,35.0,2022-08-29,2022.0,25,3,0,0,...,186,22,63,0,2,0,25,3,None,None
1,2022-09-02,9.0,September,35.0,2022-08-29,2022.0,25,3,0,0,...,186,22,63,0,2,0,50,6,None,None
2,2022-09-03,9.0,September,35.0,2022-08-29,2022.0,25,10,0,0,...,186,22,63,0,2,0,75,16,None,None
3,2022-09-04,9.0,September,35.0,2022-08-29,2022.0,36,6,0,0,...,186,22,63,0,2,0,111,22,None,None
4,2022-09-05,9.0,September,36.0,2022-09-05,2022.0,17,11,1,0,...,186,38,63,7,2,0,128,33,1,None


In [4]:
df #проверим, что получили все данные, которые хотели

,date,month,month_title,week,first_day_of_week,year,daily_visits,account_created,application_created,games,...,account_created_by_month,account_created_by_week,application_created_by_month,application_created_by_week,games_by_month,games_by_week,visits_total_by_date,account_created_total_by_date,application_created_total_by_date,games_total_by_date
0,2022-09-01,9.0,September,35.0,2022-08-29,2022.0,25,3,0,0,...,186,22,63,0,2,0,25,3,None,None
1,2022-09-02,9.0,September,35.0,2022-08-29,2022.0,25,3,0,0,...,186,22,63,0,2,0,50,6,None,None
2,2022-09-03,9.0,September,35.0,2022-08-29,2022.0,25,10,0,0,...,186,22,63,0,2,0,75,16,None,None
3,2022-09-04,9.0,September,35.0,2022-08-29,2022.0,36,6,0,0,...,186,22,63,0,2,0,111,22,None,None
4,2022-09-05,9.0,September,36.0,2022-09-05,2022.0,17,11,1,0,...,186,38,63,7,2,0,128,33,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2023-01-29,1.0,January,4.0,2023-01-23,2023.0,25,6,1,1,...,215,40,133,33,63,17,4015,998,501,170
151,2023-01-30,1.0,January,5.0,2023-01-30,2023.0,29,10,6,2,...,215,26,133,11,63,9,4044,1008,507,172
152,2023-01-31,1.0,January,5.0,2023-01-30,2023.0,20,7,2,4,...,215,26,133,11,63,9,4064,1015,509,176
153,2023-02-01,2.0,February,5.0,2023-01-30,2023.0,27,6,3,3,...,9,26,3,11,3,9,4091,1021,512,179


## Построим воронку 
### Воронка позволит нам проанализировать эффективность регистрации пользователей и посещения квестов

In [15]:
visits = df['daily_visits'].sum()
acc_created = df['account_created'].sum()
app_created = df['application_created'].sum()
games = df['games'].sum()

In [21]:
from plotly import graph_objects as go

fig = go.Figure(go.Funnel(
    y = ["Посетили сайт", "Зарегистрировали аккаунт", "Оставили заявку", "Пришли на игру"],
    x = [int(visits), int(acc_created), int(app_created), int(games)],
                textposition = "inside",
    textinfo = "value+percent initial"))

fig.show()

#### Судя по воронке из общего количества пользователей, посетивших наш сайт:
- 25% пользователей зарегистрировались 
- 12,5% пользователей оставили заявку на посещение квеста
- 4,4% пользователей пришли на игру

## Построим графики и посмотрим на динамику просмотров,регистраций,заявок и посещений квестов

In [7]:
# импортируем библиотеки для построения графиков
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [8]:
df.columns # вывожу табличку, чтоб скопировать отсюда названия столбцов, которые мне нужны

Index(['date', 'month', 'month_title', 'week', 'first_day_of_week', 'year',
       'daily_visits', 'account_created', 'application_created', 'games',
       'visits_by_month', 'visits_by_week', 'account_created_by_month',
       'account_created_by_week', 'application_created_by_month',
       'application_created_by_week', 'games_by_month', 'games_by_week',
       'visits_total_by_date', 'account_created_total_by_date',
       'application_created_total_by_date', 'games_total_by_date'],
      dtype='object')

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['month_title'], y=df['visits_by_month'],mode='lines+markers',  name='Посетили сайт'))
fig.add_trace(go.Scatter(x=df['month_title'], y=df['account_created_by_month'],mode='lines+markers', name='Зарегистрировали аккаунт'))
fig.add_trace(go.Scatter(x=df['month_title'], y=df['application_created_by_month'],mode='lines+markers', name='Оставили заявку'))
fig.add_trace(go.Scatter(x=df['month_title'], y=df['games_by_month'],mode='lines+markers', name='Пришли на игру'))
fig.update_layout(title="Динамика посетителей/регистраций/заявок/игр по месяцам",
                  xaxis_title="Месяц",
                  yaxis_title="Количество",
                  legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=50, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="Месяц: %{x}<br>Количество: %{y}")
fig.show()

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['first_day_of_week'], y=df['visits_by_week'],mode='lines+markers',  name='Посетили сайт'))
fig.add_trace(go.Scatter(x=df['first_day_of_week'], y=df['account_created_by_week'],mode='lines+markers', name='Зарегистрировали аккаунт'))
fig.add_trace(go.Scatter(x=df['first_day_of_week'], y=df['application_created_by_week'],mode='lines+markers', name='Оставили заявку'))
fig.add_trace(go.Scatter(x=df['first_day_of_week'], y=df['games_by_week'],mode='lines+markers', name='Пришли на игру'))
fig.update_layout(title="Динамика посетителей/регистраций/заявок/игр по неделям",
                  xaxis_title="Дата",
                  yaxis_title="Количество",
                  legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=50, b=0))
week_number = df['week']
fig.update_traces(hoverinfo="all", hovertemplate="Первый день недели: %{x}<br>Количество: %{y}")
fig.show()

### Если судить по этим двум графикам выше, то может показаться, что количество посетителей/регистраций/заявок/посещенных игр стабильно, но давайте вглянем на общую динамику наших показателей

#### P.S. февраль можно не учитывать, т.к. в БД данные только за первые 3 дня февраля.

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'], y=df['visits_total_by_date'],mode='lines',  name='Посетили сайт'))
fig.add_trace(go.Scatter(x=df['date'], y=df['account_created_total_by_date'],mode='lines', name='Зарегистрировали аккаунт'))
fig.add_trace(go.Scatter(x=df['date'], y=df['application_created_total_by_date'],mode='lines', name='Оставили заявку'))
fig.add_trace(go.Scatter(x=df['date'], y=df['games_total_by_date'],mode='lines', name='Пришли на игру'))
fig.update_layout(title="Общая динамика посетителей/регистраций/заявок/игр",
                  xaxis_title="Дата",
                  yaxis_title="Количество",
                  legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=0, r=0, t=50, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="Дата: %{x}<br>Количество: %{y}")
fig.show()

### По графику может показаться, что разрыв между посетившими наш сайт и посетившими игру с каждым месяцем увеличивается, но если посчитать, то получим следующие данные:
- Первый месяц (1 октября):
    - Посетителей - 804 (100%)
    - Зарегистрированных аккаунтов - 195 (24%)
    - Оставивших заявку на квест - 65 (8%)
    - Посетивших игру - 2 (0.24%)
    
- Середина ноября (15 ноября):
    - Посетителей - 804 (100%)
    - Зарегистрированных аккаунтов - 499 (24,8%)
    - Оставивших заявку на квест - 229 (11,39%)
    - Посетивших игру - 44 (2,19%)
    
- Итоговые данные (2 февраля):
    - Посетителей - 4096 (100%)
    - Зарегистрированных аккаунтов - 1024 (25%)
    - Оставивших заявку на квест - 512 (12,5%)
    - Посетивших игру - 179 (4,37%)
    
### Таким образом, у нас с каждым месяцем увеличиваются показатели, что говорит об успешности наших квестов, но имея эти данные, мы можем подумать над тем как увеличить процент регистраций,заявок и посещений игр.


## Если вам нравятся дашборды в PowerBI и есть желание понажимать на фильтры и посмотреть как будут вести себя данные, то переходите по ссылке 

### [Ссылка](https://app.powerbi.com/view?r=eyJrIjoiMmVkMDIxYzUtMGY2MS00ZDcwLWFmY2EtZTI3ZTU4YTQ5ZmMxIiwidCI6IjhiYzM0YTk5LTYzMWMtNDhlMi04NjM4LTRiMzg0YmFmOTI3MCIsImMiOjl9)